## Init.

In [1]:
# pip install mlforecast statsforecast lightgbm xgboost 

In [2]:
print("\n #######################################################################\n"
"\n # EDIT CONFIG BELOW \n"
"\n ############################################################################ \n")


 #######################################################################

 # EDIT CONFIG BELOW 

 ############################################################################ 



In [ ]:
############## CONFIGS #################################################################
import datetime
import os
category='axbx' #mention category cut of input file #awbw #axbx
train_end_date='2024-09-02' #train end 
cycle_end_date_inp='2024-09-30' #date of W+2 
#keep all files in the below folder (inputs,vhr output file, and expect outputs from here to be saved in this folder declared)
formatted_category = '_'.join(list(category))


root=os.path.dirname(os.getcwd())
folder_path =os.path.join(root,'input','O9_weekly','data')
input_output_data_folder_directory=rf"C:\Users\{os.getlogin()}\OneDrive - <>\finandops_pnl_bottomup\input\O9_weekly\data"
input_output_data_folder_directory=folder_path
############################################################################################


actuals_file_name = rf"actuals_{formatted_category}.csv" #volume inputfile
univariate_op_file_name =rf"univar_sous_{category}.csv"  #output file name to be saved into same loc
multivariate_op_file_name = rf"MULTI_SOUS_{category}.csv" #output file name to be saved into same loc
vhr_op_path_path=rf"{category}.csv" #load vhr output from here
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
final_op_name = rf"{category}_FORGE_FORECAST_{timestamp}.xlsx"


print(
"\n #######################################################################\n"
"\n # NO EDITS REQUIRED FROM BELOW LINE ONWARDS---SAFE TO RUN IN DEFAULT MODE ---AVOID CHANGING CELL POSITIONS \n"
"\n ############################################################################ \n")



 #######################################################################

 # NO EDITS REQUIRED FROM BELOW LINE ONWARDS---SAFE TO RUN IN DEFAULT MODE ---AVOID CHANGING CELL POSITIONS 

 ############################################################################ 



In [ ]:
# ----------------------------------------
# Importing necessary libraries and modules
# ----------------------------------------

# Data handling and manipulation
import pandas as pd
import numpy as np
import random
from datetime import datetime

# Display and visualization
from IPython.display import display

# Forecasting libraries and models
from statsforecast import StatsForecast
from statsforecast.utils import ConformalIntervals  
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences

# Statistical models
from statsforecast.models import (
    GARCH, 
    Naive, 
    SeasonalNaive, 
    HistoricAverage, 
    RandomWalkWithDrift, 
    CrostonClassic as Croston,
    CrostonOptimized,
    ADIDA,
    IMAPA,
    TSB,
    ARIMA,
    SeasonalExponentialSmoothing, 
    HoltWinters,
    MSTL,
    DynamicOptimizedTheta as DOT,
    AutoETS,
    AutoARIMA,
    AutoTheta,
    AutoCES,
    AutoMFLES,
    AutoTBATS,
    ARCH
)

# Machine Learning models
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor, ElasticNet
from sklearn.kernel_ridge import KernelRidge
from utilsforecast.feature_engineering import fourier

# Evaluation metrics
from sklearn.metrics import mean_absolute_percentage_error
from utilsforecast.losses import mae, mape  # Mean Absolute Error

# Utility imports
# from wsgiref.handlers import format_date_time
from numba import njit
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean
import os

# Suppressing warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

import warnings
warnings.filterwarnings("ignore")

import warnings
import sys

warnings.filterwarnings("ignore")  # Ignore all warnings
if not sys.warnoptions:
    import os
    os.environ["PYTHONWARNINGS"] = "ignore"  # Suppress warnings at the environment level

/opt/anaconda3/envs/vhr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Recipe: Univariate

In [ ]:
#dont change
def evaluate_cv(df, metric):
    """
    Evaluate the best performing models for each time series, scored against a given metric (e.g., Mean Absolute Error).
    
    """
    models = df.columns.drop(['unique_id', 'ds', 'y', 'cutoff']).tolist()
    evals = metric(df, models=models)
    evals['best_model'] = evals[models].idxmin(axis=1)
    return evals

def get_best_model_forecast(forecasts_df, evaluation_df):
    """
    Get the best performing model's forecast for each time series.
    
    """
    with_best = forecasts_df.merge(evaluation_df[['unique_id', 'best_model']])
    res = with_best[['unique_id', 'ds']].copy()
    for suffix in ('', '-lo-95', '-hi-95'):
        res[f'best_model{suffix}'] = with_best.apply(lambda row: row[row['best_model'] + suffix], axis=1)
    return res


# path =  input_output_data_folder_directory + actuals_file_name
# out_1 = input_output_data_folder_directory + univariate_op_file_name
# out_2 = input_output_data_folder_directory + multivariate_op_file_name
# vhr_op_path=input_output_data_folder_directory + vhr_op_path_path
# final_output_path= input_output_data_folder_directory + final_op_name
path=os.path.join(input_output_data_folder_directory,actuals_file_name)
out_1=os.path.join(input_output_data_folder_directory,univariate_op_file_name)
out_2=os.path.join(input_output_data_folder_directory,multivariate_op_file_name)
vhr_op_path=os.path.join(input_output_data_folder_directory,vhr_op_path_path)
final_output_path=os.path.join(input_output_data_folder_directory,final_op_name)
df = pd.read_csv(path)
cycle_end_date= pd.to_datetime(cycle_end_date_inp)- pd.Timedelta(days=1) 
print("Paths verify:\n",path.split(root)[-1],"\n",out_1.split(root)[-1],"\n",out_2.split(root)[-1],"\n",vhr_op_path.split(root)[-1])

Paths verify:
 /input/O9_weekly/data/actuals_a_x_b_x.csv 
 /input/O9_weekly/data/univar_sous_axbx.csv 
 /input/O9_weekly/data/MULTI_SOUS_axbx.csv 
 /input/O9_weekly/data/axbx.csv


In [11]:
df['Date']=pd.to_datetime(df['Date'])
n=df.Lane.nunique()
selected_uids = np.random.choice(df['Lane'].unique(), n, replace=False)

df = df[df['Lane'].isin(selected_uids)]
print(f"Data from {df['Date'].min().date()} to {df['Date'].max().date()} for {df['Lane'].nunique()} time-series")

# df.info()
df['ds']=df['Date']
df['y']=df['Actuals']
df['unique_id']=df['Lane']
df = df[['ds','y','unique_id']]
# df.head(2)


Data from 2020-12-28 to 2024-09-30 for 2092 time-series


In [ ]:
############################## SET MODEL LIST HERE ######################################################################
df=df[df['ds']<=f'{train_end_date}']
random_uids = random.choices(df['unique_id'].unique(), k=10)

# StatsForecast.plot(df, unique_ids=random_uids)
#dont change if not required - keep defaults
season_length = 52 # 52 weeks in 1 year
intervals = ConformalIntervals(h=4, n_windows=2)

mstl = MSTL(
    season_length=[4, 13, 52], # seasonalities of the time series - montlhy, quarterly and annual
    trend_forecaster=AutoARIMA() # model used to forecast trend
)

season_length_mfles = [4, 13, 52]
config = {
    'changepoints': [True, False],
    'seasonality_weights': [True, False],
    'smoother': [True, False],
    'ma': [52, 52/4, int(min(season_length_mfles)), int(min(season_length_mfles)/2), None],
    'seasonal_period': [None, season_length_mfles],
    'rs_lr': [.2],
    }
mfles = AutoMFLES(
      season_length = season_length_mfles,
      test_size = 4,
      n_windows=2,
      metric = 'mae', 
      config= config
      )

# Create a list of models and instantiation parameters
models = [
    HoltWinters(),
    Croston(),
    CrostonOptimized(),
    HistoricAverage(),
    DOT(season_length = season_length),
    SeasonalExponentialSmoothing(season_length = season_length, alpha = 0.1, prediction_intervals = intervals),
    ADIDA(prediction_intervals = intervals),
    ARIMA(order = (1,1,1), prediction_intervals = intervals),
    mstl,
    IMAPA(),
    TSB(alpha_d = 0.2, alpha_p = 0.2, prediction_intervals = intervals),
    Naive(),
    RandomWalkWithDrift(),
    mfles,
    AutoARIMA(season_length = season_length), 
    AutoETS(season_length = season_length), 
    AutoTheta(season_length = season_length), 
    AutoCES(season_length = season_length), 
    AutoTBATS(season_length = season_length),
    GARCH (p=1, q=1, alias='GARCH', prediction_intervals = intervals),
    ARCH (p=1, alias='ARCH', prediction_intervals = intervals),
    
]

# Instantiate StatsForecast class as sf
sf = StatsForecast( 
    models = models,
    freq = 'W', # weekly data frequency
    fallback_model = SeasonalNaive(season_length=season_length),
    n_jobs = 10,
    verbose=True,

)

# df[df['unique_id']== 'COM AM_99895']
# Group by 'unique_id' and count the number of rows in each group
grouped = df.groupby('unique_id').size()

# Filter out the 'unique_id's with less than 25 occurrences
valid_ids = grouped[grouped >= 102].index

# Filter the original dataframe to only include rows with 'unique_id's that have 25 or more occurrences
df = df[df['unique_id'].isin(valid_ids)]
print("pass")



pass


In [14]:
forecasts_df = sf.forecast(df=df,h=4) # 30 - 1 hour runtime in Win, 32 GB RAM

Forecast:   0%|          | 0/1000 [Elapsed: 00:00]/opt/anaconda3/envs/vhr/lib/python3.10/site-packages/statsforecast/tbats.py:949: UserWarning: Data contains zero or negative values, disabling Box-Cox transformation.
  warnings.warn(
/opt/anaconda3/envs/vhr/lib/python3.10/site-packages/statsforecast/tbats.py:949: UserWarning: Data contains zero or negative values, disabling Box-Cox transformation.
  warnings.warn(
/opt/anaconda3/envs/vhr/lib/python3.10/site-packages/statsforecast/tbats.py:949: UserWarning: Data contains zero or negative values, disabling Box-Cox transformation.
  warnings.warn(
/opt/anaconda3/envs/vhr/lib/python3.10/site-packages/statsforecast/tbats.py:949: UserWarning: Data contains zero or negative values, disabling Box-Cox transformation.
  warnings.warn(
Forecast:   0%|          | 1/1000 [Elapsed: 00:31]/opt/anaconda3/envs/vhr/lib/python3.10/site-packages/statsforecast/tbats.py:949: UserWarning: Data contains zero or negative values, disabling Box-Cox transformatio

In [15]:
forecasts_df

,unique_id,ds,HoltWinters,CrostonClassic,CrostonOptimized,HistoricAverage,DynamicOptimizedTheta,SeasonalES,ADIDA,ARIMA,...,Naive,RWD,AutoMFLES,AutoARIMA,AutoETS,AutoTheta,CES,AutoTBATS,"GARCH(1,1)",ARCH(1)
0,COM AM_38687,2024-09-08,115.500000,96.367181,96.367587,78.539627,97.071633,36.1200,96.367587,94.591038,...,63.000,63.328125,115.500000,94.369088,88.541288,100.267018,112.766837,94.542950,108.238222,87.084290
1,COM AM_38687,2024-09-15,63.000000,96.367181,96.367587,78.539627,97.136719,27.0900,96.367587,94.434755,...,63.000,63.656250,63.000000,94.369088,87.784619,100.657002,62.999693,95.278883,-40.720210,-34.486030
2,COM AM_38687,2024-09-22,126.000000,96.367181,96.367587,78.539627,97.201333,50.4000,96.367587,94.435528,...,63.000,63.984375,126.000000,94.369088,87.027951,101.046985,77.269911,95.711542,-34.755112,-27.141978
3,COM AM_38687,2024-09-29,84.000000,96.367181,96.367587,78.539627,97.265481,58.4850,96.367587,94.435524,...,63.000,64.312500,84.000000,94.369088,86.271283,101.436969,115.541596,95.831382,-69.776893,-54.740023
4,COM AM_38693,2024-09-08,959.999999,901.372288,874.750214,614.844064,874.480694,474.0000,874.750214,903.007528,...,264.000,264.387097,959.999999,871.319454,974.989388,941.743439,945.173981,824.819970,1246.798509,737.972876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7439,FratComCDDEquat_83246,2024-09-29,108.504000,75.378842,63.517530,116.350521,64.463831,135.9144,63.517530,63.625021,...,16.560,16.375283,108.504000,63.175462,63.634195,64.463831,78.430297,38.766834,-85.752703,-82.692334
7440,FratComCDDEquat_88967,2024-09-08,333.792000,182.661311,181.825771,139.358447,185.588886,333.7920,185.511666,177.670251,...,153.216,154.123765,333.792000,174.870872,185.625382,195.441973,167.954147,191.719392,202.551452,180.146221
7441,FratComCDDEquat_88967,2024-09-15,272.592000,182.661311,181.825771,139.358447,185.588891,272.5920,185.511666,183.533242,...,153.216,155.031529,272.592000,191.845070,185.625382,197.169166,166.564255,210.909426,-76.234888,-68.750378
7442,FratComCDDEquat_88967,2024-09-22,293.616000,182.661311,181.825771,139.358447,185.588896,293.6160,185.511666,184.938915,...,153.216,155.939294,293.616000,182.807651,185.625382,198.896360,163.180954,198.285799,-65.100691,-56.911441


In [9]:
forecasts_df.to_csv(out_1) #saved for review.

In [ ]:
#####################RUN IT TO CHOOSE MODEL BASED ON CV TIME TAKING CAN SAFELY SKIP FOR NOW#################################################################
# cv_df.rename(columns = {'y' : 'actual'}, inplace = True) 
# cutoff = cv_df['cutoff'].unique()

# for k in range(len(cutoff)-4): 
#     cv = cv_df[cv_df['cutoff'] == cutoff[k]]
#     # display(f"Cutoff date: {cutoff[k]}")
#     # display(StatsForecast.plot(df, cv.loc[:, cv.columns != 'cutoff'], unique_ids=[random_uids[0]]))
# cv_df.rename(columns = {'actual' : 'y'}, inplace = True) 
# evaluation_df = evaluate_cv(cv_df, mape)
# evaluation_df['best_model'].value_counts().to_frame().reset_index()
# prod_forecasts_df = get_best_model_forecast(forecasts_df, evaluation_df)
# prod_forecasts_df['best_model-lo-95'] = prod_forecasts_df['best_model-lo-95'].clip(lower=0)
# nixtla=prod_forecasts_df[prod_forecasts_df['ds']>'2024-09-02']
# test_df = pd.read_csv(r"C:\Users\40103869\OneDrive - <>finandops_pnl_bottomup\input\O9_weekly\data\actuals_a_x_b_x.csv")
# test_df['Date']=pd.to_datetime(test_df['Date'])
# test_df = test_df[test_df['Date']=='2024-09-30']
# test_df['Date'] = test_df['Date'] - pd.Timedelta(days=1) 
# nixtla['ds']=pd.to_datetime(nixtla['ds'])
# test_df['Date']=pd.to_datetime(test_df['Date'])
# final_df = nixtla.merge(test_df,right_on=['Lane','Date'],left_on=['unique_id','ds'],how='left')

# final_df = final_df[['unique_id','ds','best_model','Actuals']]
# final_df = final_df[final_df['ds']=='2024-09-29']
# final_df.to_csv(r"C:\Users\40103869\Downloads\final_df_axbx.csv")
# lala_country = final_df.groupby('ds').sum(['Actuals', 'best_model']).reset_index()
# print("SOUS COUNTRY:",(1 - (abs(lala_country['Actuals'] - lala_country['best_model']))/(lala_country['Actuals'])))
# lala_key = final_df.groupby(['ds','unique_id']).sum(['Actuals', 'best_model']).reset_index()
# lala_key['abs_diff'] = abs(lala_key['Actuals'] - lala_key['best_model'])
# lala_key_hehe = lala_key.groupby('ds').sum(['abs_diff', 'Actuals'])
# print("SOUS SPECIAL KEY LEVEL ACC",(1 - lala_key_hehe['abs_diff']/lala_key_hehe['Actuals']))
#####################################################################################################


# ######################VHR_SOUS_MIXER#####################################################
# df=pd.read_csv(r"C:\Users\40103869\Downloads\axbx.csv",parse_dates=['date'])
# df = df[['date','key','actual','prediction']]
# sous=pd.read_csv(r"C:\Users\40103869\Downloads\final_df_axbx.csv",parse_dates=['ds'])
# df['date'] = df['date'] - pd.Timedelta(days=1) 
# df=df[df['date']=='2024-09-29']
# op=df.merge(sous,left_on=['date','key'],right_on=['ds','unique_id'],how='left')
# op['err_vhr']=abs(op['Actuals']-op['prediction'])
# op['err_sous']=abs(op['Actuals']-op['best_model'])
# op['best_'] = op[['err_sous', 'err_vhr']].min(axis=1)
# print("SKU",1- op['best_'].sum()/op['Actuals'].sum())
# coun=op.groupby('date').sum(['Actuals', 'prediction','best_model_']).reset_index()
# print("Country",(1 - (abs(coun['Actuals'] - coun['prediction']))/(coun['Actuals'])))
#############################################################################################

## Recipe Multivariate

In [11]:
df = pd.read_csv(path)
df['Date']=pd.to_datetime(df['Date'])
n=df.Lane.nunique()
selected_uids = np.random.choice(df['Lane'].unique(), n, replace=False)

df = df[df['Lane'].isin(selected_uids)]
print(f"Data from {df['Date'].min().date()} to {df['Date'].max().date()} for {df['Lane'].nunique()} time-series")

df['ds']=df['Date']
df['y']=df['Actuals']
df['unique_id']=df['Lane']
df = df[['ds','y','unique_id']]
test=df.copy()
train=test[test['ds']<=f'{train_end_date}']
transformed_df, future_df = fourier(train, freq='W', season_length=4, k=2, h=4)


Data from 2020-12-28 to 2024-09-30 for 2092 time-series


In [12]:
################################## SELECT MODEL --> TO DO: Bottom models hasnt been configured yet downstream ################################################################################
models = [
    lgb.LGBMRegressor(verbosity=-1),
    xgb.XGBRegressor(),
    # LinearRegression(),
    # SGDRegressor(),
    # KernelRidge(),
    # ElasticNet(),
    # RandomForestRegressor(random_state=0),
]

@njit
def rolling_mean_60(x):
    return rolling_mean(x, window_size=8)


fcst = MLForecast(
    models=models,
    freq='W',
    lags=[1,2,3,4,5,6,12,52],
    lag_transforms={
        1: [expanding_mean],
        7: [rolling_mean_60]
    },
    date_features=['month'],
    target_transforms=[Differences([1])],
)

# Call the fit method to train the model
fcst.fit(transformed_df,static_features=[])
predictions = fcst.predict(4,X_df=future_df) #fairly quick
test['ds']=pd.to_datetime(test['ds'])
test['ds']=test['ds']-pd.Timedelta(days=1)

In [13]:
predictions = predictions.merge(test,on=['unique_id','ds'],how='left')
predictions = predictions[predictions['ds']==f'{cycle_end_date}']
predictions = predictions.dropna()
mae_lgbm = mean_absolute_percentage_error(predictions['y'], predictions['LGBMRegressor'])
mae_xgb = mean_absolute_percentage_error(predictions['y'], predictions['XGBRegressor'])
# mae_rf = mean_absolute_percentage_error(predictions['y'], predictions['RandomForestRegressor'])
# Print the MAE for each model
print(f'MAE for LGBMRegressor: {mae_lgbm}')
print(f'MAE for XGBRegressor: {mae_xgb}')
# print(f'MAE for RandomForestRegressor: {mae_rf}')
# Choose the best model based on the lowest MAE
best_model = min(
    [('LGBMRegressor', mae_lgbm), 
     ('XGBRegressor', mae_xgb), 
     ], key=lambda x: x[1])

print(f'The best model is: {best_model[0]} with MAPE: {best_model[1]}')
predictions.to_csv(out_2)
print("######## MIGHT SEE HUGE MAPE HERE< NEED TO CHECK> BUT FEW SKUS IMPROVE DURING FINAL CUT ###########################")
########################################################################################################################


MAE for LGBMRegressor: 7262116584004549.0
MAE for XGBRegressor: 8264461780562346.0
The best model is: LGBMRegressor with MAPE: 7262116584004549.0
######## MIGHT SEE HUGE MAPE HERE< NEED TO CHECK> BUT FEW SKUS IMPROVE DURING FINAL CUT ###########################


## Result Reconciliation
###### RUN ONLY IF ALL THREE ALGOS ARE APPLIED AND SAVED TO CSV - ELSE WILL BREAK

In [136]:
############################# RESULT RECONCILIATION  #############################################################################
# here prediction column is vhr op
df=pd.read_csv(vhr_op_path,parse_dates=['date']) #OP VHR FILE
df = df[['date','key','actual','prediction']]
df['date'] = df['date'] - pd.Timedelta(days=1) 
df=df[df['date']==f'{cycle_end_date}']
forecasts_df=pd.read_csv(out_1,parse_dates=['ds']) #uni sous op
forecasts_df=forecasts_df[forecasts_df['ds']==f'{cycle_end_date}']
op=df.merge(forecasts_df,left_on=['date','key'],right_on=['ds','unique_id'],how='left')
columns_to_drop = [col for col in op.columns if "hi-95" in col]
columns_to_drop1 = [col for col in op.columns if "lo-95" in col]
op = op.drop(columns_to_drop,axis=1)
op = op.drop(columns_to_drop1,axis=1)
predictions = pd.read_csv(out_2,parse_dates=['ds'])
op=op.merge(predictions,left_on=['date','key'],right_on=['ds','unique_id'],how='left')
op.drop(['y'],axis=1,inplace=True)
# List of models
models = ['HoltWinters', 'CrostonClassic', 'CrostonOptimized', 'HistoricAverage', 
          'ARIMA', 'MSTL', 'IMAPA', 'TSB', 'Naive', 'RWD', 'AutoMFLES', 'AutoARIMA', 
          'AutoETS', 'AutoTheta','prediction','XGBRegressor','LGBMRegressor','AutoTBATS','GARCH(1,1)','ARCH(1)','CES']

# Calculate errors for each model
for model in models:
    op[f'error_{model}'] = np.abs(op['actual'] - op[f'{model}'])


# Find the best model (minimum error) for each row
op['selected_model'] = op[[f'error_{model}' for model in models]].idxmin(axis=1)
op['selected_model'] = op['selected_model'].str.replace('error_', '')

# Add the best_model column with the prediction value from the selected model
# op['best_model'] = op.apply(lambda row: row[f'{row["selected_model"]}'], axis=1)
op['best_model'] = op.apply(lambda row: max(0, row[f'{row["selected_model"]}']), axis=1) #cap negative forecast to 0

columns_to_drop2 = [col for col in op.columns if "error_" in col]
op = op.drop(columns_to_drop2,axis=1)
op= op[['date','selected_model','best_model','actual','key']]

# Print separator for clarity
print("\n********** METRICS **********\n")
# Total actuals and forecasted volume
total_actuals = op['actual'].sum()
total_forecasted = op['best_model'].sum()
# Total unique keys forecasted
total_keys = op['key'].nunique()
# Read the CSV file and filter for the cycle date
df = pd.read_csv(path)
date_df = df[df['Date'] == train_end_date]
# Actuals volume and unique keys during the cycle date
total_actuals_cycle = date_df['Actuals'].sum()
total_keys_cycle = date_df['Lane'].nunique()
# Calculate the percentage of total keys forecasted
keys_forecast_percentage = (total_keys / total_keys_cycle) * 100
# Calculate the percentage of total volume forecasted
volume_forecast_percentage = (total_actuals / total_actuals_cycle) * 100
# SKU percentage (forecast accuracy)
sku_percentage = (1 - (abs(op['best_model'] - op['actual'])).sum() / total_actuals)* 100
# Country percentage (forecast accuracy by country)
coun = op.groupby('date').sum(['actual', 'best_model']).reset_index()
country_percentage = ((1 - (abs(coun['actual'] - coun['best_model'])) / coun['actual'])) * 100
# Count of keys by selected model
model_key_counts = op.groupby('selected_model')['key'].count().sort_values()
sku_volume_missed = total_actuals_cycle - total_actuals
metrics_data = {
    'Metric': [
        'SKU Accuracy (%)',
        'Country Forecast Accuracy (%)',
        'Actuals Volume',
        'Forecasted Volume',
        'Total Keys Actuals',
        'Total Keys Forecasted',
        'Total Actuals Volume on the date',
        'Percentage of Total Keys Forecasted',
        'Percentage of Actual Volume Considered',
        'Volume missed due to SKU drop'
        
        
    ],
    'Value': [
        sku_percentage,
        country_percentage,
        total_actuals,
        total_forecasted,
        total_keys_cycle,
        total_keys,
        total_actuals_cycle,
        keys_forecast_percentage,
        volume_forecast_percentage,
        sku_volume_missed,
        
        
    ]
}

# Create DataFrame
metrics_df = pd.DataFrame(metrics_data)
# Ensure all values in the 'Value' column are of type float64
metrics_df['Value'] = metrics_df['Value'].astype('float64').round(3)
display(metrics_df)
display(model_key_counts)
# End of experiment
print("********** END OF EXPERIMENT **********")


# Save to Excel
output_file_sheet = final_output_path
with pd.ExcelWriter(output_file_sheet, engine='xlsxwriter') as writer:
    op.to_excel(writer,sheet_name="forecasts",index=False)
    metrics_df.to_excel(writer, sheet_name='Metrics', index=False)

print(f"Metrics saved to {output_file_sheet}")

############################################## END OF RECONCILIATION ###########################################################################



********** METRICS **********



,Metric,Value
0,SKU Accuracy (%),76.260
1,Country Forecast Accuracy (%),95.434
2,Actuals Volume,701525.425
3,Forecasted Volume,669496.915
4,Total Keys Actuals,1993.000
5,Total Keys Forecasted,1606.000
6,Total Actuals Volume on the date,793246.698
7,Percentage of Total Keys Forecasted,80.582
8,Percentage of Actual Volume Considered,88.437
9,Volume missed due to SKU drop,91721.274


selected_model
IMAPA                 3
GARCH(1,1)            4
ARCH(1)              16
CrostonOptimized     23
ARIMA                25
CrostonClassic       30
AutoTheta            37
TSB                  52
AutoARIMA            55
AutoETS              64
AutoTBATS            92
CES                  98
RWD                 104
XGBRegressor        124
Naive               134
HistoricAverage     151
LGBMRegressor       154
prediction          190
HoltWinters         250
Name: key, dtype: int64

********** END OF EXPERIMENT **********
Metrics saved to C:\Users\40103869\OneDrive - Anheuser-Busch InBev\ForecastingTech\o9_vhr\finandops_pnl_bottomup\input\O9_weekly\data\axbx_FORGE_FORECAST_20250322_005142.xlsx


In [137]:
# Compute the Bias (difference between best_model and actual)
op['bias'] = op['best_model'] - op['actual']
op['bias'].sum()/op['actual'].sum()*100

-4.565552173895692

#### ALTERNATIVE RESULT RECON

In [ ]:
# ################################ RESULT RECONCILATION [DEPRECATED] #####################################################
# # sous not broken down
# import datetime
# df=pd.read_csv(rf"C:\Users\40103869\Downloads\{category}.csv",parse_dates=['date']) #vhr op
# df = df[['date','key','actual','prediction']]
# sous=pd.read_csv(rf"C:\Users\40103869\Downloads\univar_sous_{category}.csv",parse_dates=['ds']) #uni sous op
# df['date'] = df['date'] - pd.Timedelta(days=1) 
# predictions = pd.read_csv(rf"C:\Users\40103869\Downloads\MULTI_SOUS_{category}.csv") #multi sous
# df=df[df['date']==f'{cycle_end_date}']
# op=df.merge(sous,left_on=['date','key'],right_on=['ds','unique_id'],how='left')
# op=op.merge(predictions,left_on=['date','key'],right_on=['ds','unique_id'],how='left')
# op['err_vhr']=abs(op['Actuals']-op['prediction'])
# op['err_sous']=abs(op['Actuals']-op['best_model'])
# op[f'err_{best_model[0]}']=abs(op['Actuals']-op[f'{best_model[0]}'])
# op['best_'] = op[['err_sous', 'err_vhr',f'err_{best_model[0]}']].min(axis=1)

# conditions = [
#     op['err_sous'] == op[['err_sous', 'err_vhr', f'err_{best_model[0]}']].min(axis=1),
#     op['err_vhr'] == op[['err_sous', 'err_vhr', f'err_{best_model[0]}']].min(axis=1),
#     op[f'err_{best_model[0]}'] == op[['err_sous', 'err_vhr', f'err_{best_model[0]}']].min(axis=1)
# ]

# choices = ['Sous', 'VHR', f'{best_model[0]}']

# # Apply the condition to the 'selected_model' column
# op['selected_model'] = np.select(conditions, choices)

# #  Store the best forecast (final result) in the 'finalresu' column
# op['SELECTED_FORECAST'] = op.apply(lambda row: row['best_model'] if row['selected_model'] == 'Sous' else
#                            (row['prediction'] if row['selected_model'] == 'VHR' else row[best_model[0]]), axis=1)


# print("SKU",1- op['best_'].sum()/op['Actuals'].sum())
# coun=op.groupby('date').sum(['Actuals', 'SELECTED_FORECAST',]).reset_index()
# print("Country",(1 - (abs(coun['Actuals'] - coun['SELECTED_FORECAST']))/(coun['Actuals'])))
# op=op[['date','key','selected_model','SELECTED_FORECAST','actual']]
# op.to_csv(f'{category}_FORGE_FORECAST'+ f'{str(datetime.datetime.now())}'+'.csv')

# ############################## END OF RECONCILATION #############################################################